In [1]:
import os
from datetime import datetime
import requests
import boto3

# Configuration
BASE_URL = "https://api.tmb.cat/v1/"
ENDPOINT = "planner/plan"
BUCKET = "tmbinfo"
BUCKET_FOLDER = "routes_from_api"

def get_journey_plan(home_location, work_location, tmb_app_id, tmb_app_key):
    """
    Retrieve the journey plan from TMB API.

    Parameters:
        home_location (dict): The starting location with 'latitude' and 'longitude'.
        work_location (dict): The destination location with 'latitude' and 'longitude'.
        tmb_app_id (str): TMB API application ID.
        tmb_app_key (str): TMB API application key.

    Returns:
        dict: Journey plan data from the API response.
    """
    from_place = f"{home_location['latitude']},{home_location['longitude']}"
    to_place = f"{work_location['latitude']},{work_location['longitude']}"
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M")
    mode = ['TRANSIT', 'WALK']

    params = {
        'app_key': tmb_app_key,
        'app_id': tmb_app_id,
        'fromPlace': from_place,
        'toPlace': to_place,
        'date': date,
        'time': time,
        'mode': ','.join(mode),
        'showIntermediateStops': True
    }

    response = requests.get(f"{BASE_URL}{ENDPOINT}", params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

def store_journey_plan(journey_plan):
    """
    Store the journey plan in an S3 bucket, in the folder routes_from_api.

    Parameters:
        journey_plan (dict): The journey plan data to store.
    """
    try: 
        s3 = boto3.client('s3')
        key = f"{BUCKET_FOLDER}/journey_plan_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.json"
        s3.put_object(Bucket=BUCKET, Key=key, Body=journey_plan)

        return {
            "statusCode": 200,
            "message": "Journey plan successfully stored",
        }
    except Exception as e:
        print(e)
        raise e

def lambda_handler(event, context):
    """
    Lambda function handler for retrieving and storing journey plans.

    Parameters:
        event (dict): Event data passed by Lambda.
        context (object): Context object provided by Lambda.

    Returns:
        dict: API Gateway Lambda Proxy Output Format.
    """
    try:
        tmb_app_id = "8de5b882"
        tmb_app_key = "a9e0b1a36bd4417ffd5a815b186d043d"

        home_location = {
            'latitude': 41.423043,
            'longitude': 2.184006
        }

        work_location = {
            'latitude': 41.406232,
            'longitude': 2.192273
        }

        journey_plan = get_journey_plan(home_location, work_location, tmb_app_id, tmb_app_key)
        store_journey_plan(journey_plan)

        return {
            "statusCode": 200,
            "message": "Journey plan successfully retrieved",
            "body": journey_plan,
        }
    
    except requests.RequestException as e:
        print(e)
        raise e

In [2]:
import uuid

def extract_routes(data):
    routes = []

    if 'plan' in data and 'itineraries' in data['plan']:
        for itinerary in data['plan']['itineraries']:
            route = {
                'id': str(uuid.uuid4()),  # Unique identifier
                'duration': itinerary.get('duration'),
                'transfers': itinerary.get('transfers'),
                'legs': [],
                'modes': set()  # Use a set to collect unique modes
            }
            
            for leg in itinerary.get('legs', []):
                leg_info = {
                    'mode': leg.get('mode'),
                    'start_time': leg.get('startTime'),
                    'end_time': leg.get('endTime'),
                    'from': leg.get('from', {}).get('name'),
                    'to': leg.get('to', {}).get('name'),
                    'route': leg.get('route'),
                    'distance': leg.get('distance'),
                    'agency': leg.get('agencyName')
                }
                route['legs'].append(leg_info)
                route['modes'].add(leg.get('mode'))  # Add the mode to the set
            
            route['modes'] = list(route['modes'])  # Convert the set to a list
            routes.append(route)
    
    return routes

# Extract route information
routes = extract_routes(journey_plan)


In [3]:
import pandas as pd

routes_df = pd.json_normalize(routes, 'legs', ['id', 'duration', 'transfers', 'modes'])

routes_df.sort_values(['id', 'start_time'])

,mode,start_time,end_time,from,to,route,distance,agency,id,duration,transfers,modes
7,WALK,1722935275000,1722935601000,Origin,Meridiana - La Sagrera,,347.73,None,0023f788-4463-440a-9d2c-e8ad48141c56,1467,0,"[WALK, BUS]"
8,BUS,1722935616000,1722936064000,Meridiana - La Sagrera,Meridiana - Aragó,<M> Clot / Ciutat Meridiana,1551.03,TMB,0023f788-4463-440a-9d2c-e8ad48141c56,1467,0,"[WALK, BUS]"
9,WALK,1722936079000,1722936742000,Meridiana - Aragó,Destination,,694.73,None,0023f788-4463-440a-9d2c-e8ad48141c56,1467,0,"[WALK, BUS]"
4,WALK,1722935225000,1722935415000,Origin,Pl dels Indians,,140.02,None,1abdf27d-6df9-4363-90a5-567903d24900,1512,0,"[WALK, BUS]"
5,BUS,1722935430000,1722935940000,Pl dels Indians,Espronceda - Gran Via,Pg. Marítim / Canyelles,2098.52,TMB,1abdf27d-6df9-4363-90a5-567903d24900,1512,0,"[WALK, BUS]"
6,WALK,1722935955000,1722936737000,Espronceda - Gran Via,Destination,,848.61,None,1abdf27d-6df9-4363-90a5-567903d24900,1512,0,"[WALK, BUS]"
1,WALK,1722935125000,1722935527000,Origin,La Sagrera,,552.96,None,68c88c27-5c5e-468a-a421-2141ae0de023,1573,0,"[WALK, SUBWAY]"
2,SUBWAY,1722935542000,1722935700000,La Sagrera,Clot,Hospital de Bellvitge - Fondo,1357.41,TMB,68c88c27-5c5e-468a-a421-2141ae0de023,1573,0,"[WALK, SUBWAY]"
3,WALK,1722935715000,1722936698000,Clot,Destination,,1008.75,None,68c88c27-5c5e-468a-a421-2141ae0de023,1573,0,"[WALK, SUBWAY]"
0,WALK,1722935100000,1722937132000,Origin,Destination,,2437.03,None,edf72979-513c-46e9-aed1-c78f36bb630a,2032,0,[WALK]
